In [1]:
#率変化(MGG(ルーレット))
import random
import math
import numpy as np
import pandas as pd
type(np.int32(np.atleast_1d(np.int32(0))))
from operator import attrgetter
import matplotlib.pyplot as plt
from IPython.display import clear_output
print('ok')

ok


In [2]:
writing=open("GA_for_BLUEJAY_2峰.csv","w",encoding="UTF-8")
writing.write("H1,theta1,lambda1,omega1,H2,theta2,lambda2,omega2,Error")

55

In [ ]:
wavedate=pd.read_csv("1 NYK BLUE-JAY/BLUE-JAYfrom20161201to20190129.csv", encoding="shift_jis",engine="python")#
wavedata=wavedate.values[:,16:25*36+16]
#d_df= np.array([np.reshape(x[0+16:input_shape_rows*input_shape_columns+16],(input_shape_rows,input_shape_columns)) for x in wavedate.values])
#[[float(wavedate.iloc[i][j+1])for j in range(36)] for i in range(25)]#wavedate.values[0:25,1:37]
#wavedata=[[float(wavedate.iloc[i][j])for j in range(36)] for i in range(25)]
freq=np.array([0.0445953,0.0486315,0.053033,0.0578329,0.0630672,0.0687753,0.075,0.0817881,0.0891906,0.097263,0.106066,0.1156658,0.1261345,0.1375506,0.15,0.1635762,0.1783811,0.194526,0.2121321,0.2313317,0.252269,0.2751013,0.3000001,0.3271524,0.3567623])
print(np.shape(wavedata))
print(wavedata[0][0])

def main():
    
    n_gene   = 64   # The number of genes.
    n_ind    = 64  #the number of individuals in a population.
    CXPB     = 0.85   # The probability of crossover.
    MUTPB    = 1.0   # The probability of individdual mutation.
    NGEN     = 10000   # The number of generation loop.
    n_parents=2     #非復元抽出の個体数
    MUTINDPB = 0.0050  # The probability of gene mutation.
    CXPB_MAX =0.900   #交叉率の最大値
    CXPB_MIN =0.200   #交叉率の最小値
    M_MAX    =0.1#10.0/n_gene  #突然変異率の最大値
    M_MIN    =0.01#1.0/n_gene/2  #突然変異率の最小値
    f        =15
    
    for i in range(len(wavedata)):
        global sea
        sea=i
        print(str(len(wavedata))+"番中　"+str(i+1)+"番目")
        random.seed()
        Loss_Function=[]
    
        """ Step1 : 母集団作成."""
        pop = create_pop(n_ind, n_gene)
        set_fitness(evalOneMax, pop)  #各個体について評価
        fit_sum = keisan(n_ind,pop)  #適応度の合計値を計算
        best_ind = min(pop, key=attrgetter("fitness"))  #最良個体の適応度

        # --- 世代分の繰り返し.
        #print("Generation loop start.")
        #print("Generation: 0. Best fitness: " + str(best_ind.fitness))
        for g in range(NGEN):     
            """ Step2 : 選択."""
            parents,pop = randomselection(n_ind,pop,n_parents)
            fit_sum=fit_sum-parents[0].fitness-parents[1].fitness
            children = parents.copy()
  
            """ Step3 : 交叉."""
            crossover = []
            for child1, child2 in zip(children[::2], children[1::2]):
                if random.random() < CXPB:
                    child1, child2 = cxTwoPointCopy(child1, child2)
                    child1.fitness = None
                    child2.fitness = None
                crossover.append(child1)
                crossover.append(child2)
            children = crossover[:]

            """ Step4 : 突然変異."""
            mutant = []
            for mut in (children):
                if random.random() < MUTPB:
                    mut = mutFlipBit(mut, indpb=MUTINDPB)
                    mut.fitness = None
                mutant.append(mut)

            children = mutant[:]
            set_fitness(evalOneMax, children)

        
            """Step5 : 生存選択"""
            family = children+parents
            offspring = []
        
            family_sorted = sorted(family, key=attrgetter("fitness")) #ここで，誤差が小さい順に並べる
            offspring=roulette(family_sorted[1:],family_sorted[3].fitness)
            offspring+=family_sorted[:1]

        
            """ 次世代を現世代に更新."""
            pop = pop + offspring
            fit_sum = fit_sum+offspring[0].fitness+offspring[1].fitness
            fitness_mean =fit_sum/n_ind #,fit_var
            """ Print best fitness in the population."""
            best_ind = min(pop, key=attrgetter("fitness"))
            f_max = best_ind.fitness
            CXPB,MUTINDPB = adjustrating(CXPB,MUTINDPB,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean)
        
            #Loss_Function.append(best_ind.fitness)
        
            #if (g+1) % 100 == 0:
                #clear_output(wait=True)
                #plt.close()
                #print("Generation: " + str(g+1) + ". Best fitness: " + str(best_ind.fitness)+ " mean: "+str(int(fitness_mean))+" MU: "+str(MUTINDPB))#+" CXPB: "+str(CXPB))  #+" variance: " + str(int(fit_var))
                #plt.plot(Loss_Function)
                #plt.show()
        lambda_1,theta_1,omega_1, Hv_1,lambda_2,theta_2,omega_2,Hv_2=get_parameter_from_ind(best_ind)   
        #print("Generation loop ended. The best individual: ")
        #print(best_ind)
        #print("Hv1",Hv_1,"theta1",theta_1,"lambda1",lambda_1,"w1",omega_1)
        #print("Hv2",Hv_2,"theta2",theta_2,"lambda2",lambda_2,"w2",omega_2)
        if omega_1 < omega_2:
            writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1*math.pi/180))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+",")
            writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2*math.pi/180))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+",")
        elif omega_1 == omega_2 :
            if Hv_1 >= Hv_2 :
                writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1*math.pi/180))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+",")
                writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2*math.pi/180))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+",")
            else:
                writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2*math.pi/180))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+",")
                writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1*math.pi/180))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1)))
        else:
            writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2*math.pi/180))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+",")
            writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1*math.pi/180))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+",")    
        writing.write(str('%03.8f' %(best_ind.fitness)))
        writing.write('\n')

    
    
class Individual(np.ndarray):
    """Container of a individual."""
    fitness = None
    
    def __new__(cls, a):
        return np.asarray(a).view(cls)

def create_ind(n_gene):
    """Create a individual."""
    return Individual([np.random.randint(0,2) for i in range(n_gene)])

def create_pop(n_ind, n_gene):
    """Create a population."""
    pop = []
    for i in range(n_ind):
        ind = create_ind(n_gene)
        pop.append(ind)
    return pop

def set_fitness(eval_func, pop):
    """Set fitnesses of each individual in a population."""
    for i, fit in zip(range(len(pop)), map(eval_func, pop)):
        pop[i].fitness = fit

def spectrum(lambda_1,theta_1,omega_1, Hv_1,s_1,theta_func,omega):
    spectrum_11=0.250*((4.0*lambda_1+1.0)/4.0*omega_1**4.0)**lambda_1
    spectrum_12=math.gamma(lambda_1)
    spectrum_16=Hv_1**2.0
    spectrum_13=omega**(4.0*lambda_1+1.0)
    spectrum_14=2.0**(2.0*s_1-1.0)*(math.gamma(s_1+1.0))**2.0/math.pi/math.gamma(2.0*s_1+1.0)
    spectrum_15=abs((math.cos((theta_func-theta_1)/2.0*math.pi/180.0)))**(2.0*s_1)*math.exp(-(4.0*lambda_1+1.0)/4.0*(omega_1/omega)**4.0)
    spectrum_1=spectrum_11/spectrum_13*spectrum_14*spectrum_15*spectrum_16/spectrum_12
    return spectrum_1
    
    
def evalOneMax(ind):
    """Objective function."""
    errorfunc=0.0
    lambda_1,theta_1,omega_1, Hv_1,lambda_2,theta_2,omega_2,Hv_2 = get_parameter_from_ind(ind)
    S_max1=def_S_max(Hv_1,omega_1)
    S_max2=def_S_max(Hv_2,omega_2)
   
    for i in range(25):
        omega=freq[i]*2*math.pi
        
        """1峰目"""
        if omega<=omega_1:
            s_1=(S_max1*omega**5/omega_1**5)
        else:
            s_1=(S_max1*omega**(-2.5)/omega_1**(-2.5))                           
 
        """2峰目"""
        if omega<=omega_2:
            s_2=(S_max2*omega**5/omega_2**5)
        else:
            s_2=(S_max2*omega**(-2.5)/omega_2**(-2.5))  
        
        for j in range(36):
            theta_func=j*10#wavedate[0][j+1]

            """1峰目"""
            if omega<=omega_1:
                s_1=(S_max1*omega**5/omega_1**5)
            else:
                s_1=(S_max1*omega**(-2.5)/omega_1**(-2.5))                           
            
            """2峰目"""
            if omega<=omega_2:
                s_2=(S_max2*omega**5/omega_2**5)
            else:
                s_2=(S_max2*omega**(-2.5)/omega_2**(-2.5))      
            
            errorfunc +=((wavedata[sea][36*i+j]-2*math.pi*(spectrum(lambda_1,theta_1,omega_1, Hv_1,s_1,theta_func,omega)+spectrum(lambda_2,theta_2,omega_2, Hv_2,s_2,theta_func,omega))))**2.0
    return errorfunc

def adjustrating(C_p,M_p,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean):
    cross_para = 1.0/f-1/abs(f_max-fitness_mean)
    if (C_p*(1+cross_para))<=CXPB_MIN:
        C_p = CXPB_MIN
    else:
        if (C_p*(1+cross_para))<=CXPB_MAX:
            C_p = C_p*(1+cross_para)
        else:
            C_p = CXPB_MAX       
                                  
    M_para = 1/abs(f_max-fitness_mean) - 1/f
    if (M_p*(1+M_para))<=M_MIN:
        M_p = M_MIN
    else:
        if (M_p*(1+M_para))<=M_MAX:
            M_p = M_p*(1+M_para)
        else:
            M_p = M_MAX
         
    return  C_p,M_p

def randomselection(n_ind,pop,n_parents):
    chosen=[]    
    for i in range(n_parents):
        idx=random.randrange(0, n_ind-i)
        chosen.append(pop[idx])
        pop.pop(idx)
    return chosen,pop
    """chosen = []   
    eve1=random.randrange(0, n_ind, 2)
    odd1=random.randrange(1, n_ind, 2)
    chosen.append(pop[eve1])
    chosen.append(pop[odd1])
    if eve1>odd1:
        pop.pop(eve1)
        pop.pop(odd1)
    else:
        pop.pop(odd1)
        pop.pop(eve1) 
    return chosen,pop"""


def tournament(pop, n_ind, tournsize):
    """#Selection function."""
    chosen = []    
    for i in range(n_ind):
        aspirants = [random.choice(pop) for j in range(tournsize)]
        chosen.append(min(aspirants, key=attrgetter("fitness")))
    return chosen

def roulette(pop,C_max):
    """Selection function."""
    chosen=[]
    fitness_sum=0.0
    for i in range(len(pop)):
        fitness_sum += C_max- pop[i].fitness
        
    sigma_fitness = 0.0
    s = random.random()*fitness_sum
    for i in range(len(pop)):
        sigma_fitness += C_max-pop[i].fitness
        if (sigma_fitness)>=s:
            chosen.append(pop[i])
            break
    #chosens=chosen[:]
    #print("chosens"+str(len(chosens)))
    return chosen

def cxTwoPointCopy(ind1, ind2):
    """Crossover function."""
    size = len(ind1)
    tmp1 = ind1.copy()
    tmp2 = ind2.copy()
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size-1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    tmp1[cxpoint1:cxpoint2], tmp2[cxpoint1:cxpoint2] = tmp2[cxpoint1:cxpoint2].copy(), tmp1[cxpoint1:cxpoint2].copy()
    return tmp1, tmp2

def evalchild(offspring,n_ind):
    fit_mean = 0.0
    fit_var  = 0.0
    for i in range(n_ind):
        fit_mean += offspring[i].fitness/n_ind
    for k in range(n_ind):
        fit_var += (offspring[k].fitness - fit_mean)**2/n_ind
    return fit_mean, fit_var

def mutFlipBit(ind, indpb):
    """Mutation function."""
    tmp = ind.copy()
    for i in range(len(ind)):
        if random.random() < indpb:
            tmp[i] = type(ind[i])(not ind[i])
    return tmp

def keisan(n_ind,pop):
    Ex=0.0
    #Var=0.0
    for i in range(n_ind):
        Ex += pop[i].fitness
    #for i in range(n_ind):
        #Var += (pop[i].fitness-Ex)**2/n_ind
    return Ex#,Var
        

def get_parameter_from_ind(ind):
    """1峰目"""
    Hv_1     =0.10+0.020*np.sum([ind[i]*2**i for i in range(9)])#0.1<=H<=10.32
    lambda_1 =0.01+0.015*np.sum([ind[i+9]*2**i for i in range(9)])#0.01<=H<=7.675
    theta_1  =2.8125*np.sum([ind[i+18]*2**i for i in range(7)])
    omega_1  =0.35+0.015*np.sum([ind[i+25]*2**i for i in range(7)])#0.35<=omega<=2.255
    """2峰目"""
    Hv_2     =0.020*np.sum([ind[i+32]*2**i for i in range(9)])#0<=H<=10.22
    lambda_2 =0.01+0.015*np.sum([ind[i+41]*2**i for i in range(9)])#0.01<=H<=7.675
    theta_2  =2.8125*np.sum([ind[i+50]*2**i for i in range(7)])
    omega_2  =0.35+0.015*np.sum([ind[i+57]*2**i for i in range(7)])#0.35<=omega<=2.255
    return lambda_1,theta_1,omega_1, Hv_1,lambda_2,theta_2,omega_2,Hv_2

def def_S_max(H,omega_m):
    HL=H/(1.56*4*math.pi**2/omega_m**2)
    """波集中度s1の周波数依存"""
    if H==0:
        S_max=0
    elif HL<0.026:
        log_10=0.4444*math.log10(HL)+0.5849
        a=10**(log_10)
        S_max=12.99*a**(-2.5)
        
    elif HL>=0.026:
        log10_gF=(0.1507-math.sqrt(0.1507**2+4*0.005882*0.8789+4*0.005882*math.log10(HL)))/(2*0.005882)
        gF=10**log10_gF
        gT=1.37*(1-1/(1+0.008*gF**(1/3))**5)
        S_max=12.99*gT**2.5
    
    if S_max >=84:
        S_max=84
    return S_max


if __name__ == "__main__":

    main()

(14345, 900)
9.83e-11
14345番中　1番目
14345番中　2番目
14345番中　3番目
14345番中　4番目
14345番中　5番目
14345番中　6番目
14345番中　7番目
14345番中　8番目
14345番中　9番目
14345番中　10番目
14345番中　11番目
14345番中　12番目
14345番中　13番目
14345番中　14番目
14345番中　15番目
14345番中　16番目
14345番中　17番目
14345番中　18番目
14345番中　19番目
14345番中　20番目
14345番中　21番目
14345番中　22番目
14345番中　23番目
14345番中　24番目
14345番中　25番目
14345番中　26番目
14345番中　27番目
14345番中　28番目
14345番中　29番目
14345番中　30番目
14345番中　31番目
14345番中　32番目
14345番中　33番目
14345番中　34番目
14345番中　35番目
14345番中　36番目
14345番中　37番目
14345番中　38番目
14345番中　39番目
14345番中　40番目
14345番中　41番目
14345番中　42番目
14345番中　43番目
14345番中　44番目
14345番中　45番目
14345番中　46番目
14345番中　47番目
14345番中　48番目
14345番中　49番目
14345番中　50番目
14345番中　51番目
14345番中　52番目
14345番中　53番目
14345番中　54番目
14345番中　55番目
14345番中　56番目
14345番中　57番目
14345番中　58番目
14345番中　59番目
14345番中　60番目
14345番中　61番目
14345番中　62番目
14345番中　63番目
14345番中　64番目
14345番中　65番目
14345番中　66番目
14345番中　67番目
14345番中　68番目
14345番中　69番目
14345番中　70番目
14345番中　71番目
14345番中　72番目
14345番中　73番目
14345番中　74番目
14345番中　75番目
14345番中　76番目

14345番中　593番目
14345番中　594番目
14345番中　595番目
14345番中　596番目
14345番中　597番目
14345番中　598番目
14345番中　599番目
14345番中　600番目
14345番中　601番目
14345番中　602番目
14345番中　603番目
14345番中　604番目
14345番中　605番目
14345番中　606番目
14345番中　607番目
14345番中　608番目
14345番中　609番目
14345番中　610番目
14345番中　611番目
14345番中　612番目
14345番中　613番目
14345番中　614番目
14345番中　615番目
14345番中　616番目
14345番中　617番目
14345番中　618番目
14345番中　619番目
14345番中　620番目
14345番中　621番目
14345番中　622番目
14345番中　623番目
14345番中　624番目
14345番中　625番目
14345番中　626番目
14345番中　627番目
14345番中　628番目
14345番中　629番目
14345番中　630番目
14345番中　631番目
14345番中　632番目
14345番中　633番目
14345番中　634番目
14345番中　635番目
14345番中　636番目
14345番中　637番目
14345番中　638番目
14345番中　639番目
14345番中　640番目
14345番中　641番目
14345番中　642番目
14345番中　643番目
14345番中　644番目
14345番中　645番目
14345番中　646番目
14345番中　647番目
14345番中　648番目
14345番中　649番目
14345番中　650番目
14345番中　651番目
14345番中　652番目
14345番中　653番目
14345番中　654番目
14345番中　655番目
14345番中　656番目
14345番中　657番目
14345番中　658番目
14345番中　659番目
14345番中　660番目
14345番中　661番目
14345番中　662番目
14345番中　663番目
14345番

14345番中　1167番目
14345番中　1168番目
14345番中　1169番目
14345番中　1170番目
14345番中　1171番目
14345番中　1172番目
14345番中　1173番目
14345番中　1174番目
14345番中　1175番目
14345番中　1176番目
14345番中　1177番目
14345番中　1178番目
14345番中　1179番目
14345番中　1180番目
14345番中　1181番目
14345番中　1182番目
14345番中　1183番目
14345番中　1184番目
14345番中　1185番目
14345番中　1186番目
14345番中　1187番目
14345番中　1188番目
14345番中　1189番目
14345番中　1190番目
14345番中　1191番目
14345番中　1192番目
14345番中　1193番目
14345番中　1194番目
14345番中　1195番目
14345番中　1196番目
14345番中　1197番目
14345番中　1198番目
14345番中　1199番目
14345番中　1200番目
14345番中　1201番目
14345番中　1202番目
14345番中　1203番目
14345番中　1204番目
14345番中　1205番目
14345番中　1206番目
14345番中　1207番目
14345番中　1208番目
14345番中　1209番目
14345番中　1210番目
14345番中　1211番目
14345番中　1212番目
14345番中　1213番目
14345番中　1214番目
14345番中　1215番目
14345番中　1216番目
14345番中　1217番目
14345番中　1218番目
14345番中　1219番目
14345番中　1220番目
14345番中　1221番目
14345番中　1222番目
14345番中　1223番目
14345番中　1224番目
14345番中　1225番目
14345番中　1226番目
14345番中　1227番目
14345番中　1228番目
14345番中　1229番目
14345番中　1230番目
14345番中　1231番目
14345番中　1232番目
14345番中　12

In [17]:
f.close()